In [1]:
!pip install tensorflow matplotlib seaborn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls "/content/drive/MyDrive/kaggle/"




Altocumulus  Cirrocumulus  Cirrus	 Cumulus       Stratocumulus
Altostratus  Cirrostratus  Cumulonimbus  Nimbostratus  Stratus


In [ ]:
import os, shutil, random

# Source folder with cloud categories
src_dir = "/content/drive/MyDrive/kaggle"
train_dir = "/content/drive/MyDrive/cloud_split/train"
test_dir = "/content/drive/MyDrive/cloud_split/test"

os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

for category in os.listdir(src_dir):
    category_path = os.path.join(src_dir, category)
    if os.path.isdir(category_path):
        files = os.listdir(category_path)
        random.shuffle(files)
        split = int(0.8 * len(files))

        os.makedirs(os.path.join(train_dir, category), exist_ok=True)
        os.makedirs(os.path.join(test_dir, category), exist_ok=True)

        for i, file in enumerate(files):
            src = os.path.join(category_path, file)
            dst = os.path.join(train_dir if i < split else test_dir, category, file)
            shutil.copy(src, dst)


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_size = (128, 128)
batch_size = 32

train_path = "/content/drive/MyDrive/cloud_split/train"
test_path  = "/content/drive/MyDrive/cloud_split/test"

train_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    train_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    test_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)



Found 5923 images belonging to 10 classes.
Found 2134 images belonging to 10 classes.


In [ ]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(train_gen.num_classes, activation='softmax')  # 10 classes
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,305,930 (12.61 MB)

 Trainable params: 3,305,930 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Function to check and clean directories
def clean_directory(directory):
    print(f"Checking and cleaning directory: {directory}")
    for subdir in os.listdir(directory):
        subdir_path = os.path.join(directory, subdir)
        if os.path.isdir(subdir_path):
            for filename in os.listdir(subdir_path):
                filepath = os.path.join(subdir_path, filename)
                if not filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                    print(f"Removing non-image file: {filepath}")
                    os.remove(filepath)
        else:
             # If it's a file directly in the main directory, remove it
            filepath = os.path.join(directory, subdir)
            if not subdir.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                 print(f"Removing non-image file: {filepath}")
                 os.remove(filepath)

# Clean the training and testing directories
clean_directory(train_path)
clean_directory(test_path)

# Recreate the image data generators after cleaning
train_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    train_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    test_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

Checking and cleaning directory: /content/drive/MyDrive/cloud_split/train
Checking and cleaning directory: /content/drive/MyDrive/cloud_split/test
Found 5923 images belonging to 10 classes.
Found 2134 images belonging to 10 classes.


In [ ]:
from PIL import Image
import os

def remove_corrupt_images(folder):
    total = 0
    removed = 0
    for subdir, dirs, files in os.walk(folder):
        for file in files:
            filepath = os.path.join(subdir, file)
            try:
                img = Image.open(filepath)
                img.verify()  # verify image integrity
                total += 1
            except:
                os.remove(filepath)
                removed += 1
    print(f"Checked {total + removed} files. Removed {removed} corrupt images.")

# Clean both training and test folders
remove_corrupt_images("/content/drive/MyDrive/cloud_split/train")
remove_corrupt_images("/content/drive/MyDrive/cloud_split/test")


Checked 5923 files. Removed 1 corrupt images.
Checked 2134 files. Removed 0 corrupt images.


In [ ]:
from PIL import Image
import os

def remove_corrupt_images(folder):
    total = 0
    removed = 0
    for subdir, dirs, files in os.walk(folder):
        for file in files:
            filepath = os.path.join(subdir, file)
            try:
                img = Image.open(filepath)
                img.verify()  # verify image integrity
                total += 1
            except:
                os.remove(filepath)
                removed += 1
    print(f"Checked {total + removed} files. Removed {removed} corrupt images.")

# Clean both training and test folders
remove_corrupt_images("/content/drive/MyDrive/cloud_split/train")
remove_corrupt_images("/content/drive/MyDrive/cloud_split/test")

# Recreate the image data generators after cleaning
train_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    train_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    test_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

Checked 5922 files. Removed 0 corrupt images.
Checked 2134 files. Removed 0 corrupt images.
Found 5922 images belonging to 10 classes.
Found 2134 images belonging to 10 classes.


In [ ]:
import os

def remove_non_images(folder):
    allowed_exts = ['.jpg', '.jpeg', '.png']
    removed = 0
    for subdir, _, files in os.walk(folder):
        for file in files:
            ext = os.path.splitext(file)[1].lower()
            if ext not in allowed_exts:
                filepath = os.path.join(subdir, file)
                os.remove(filepath)
                removed += 1
    print(f"Removed {removed} non-image files from: {folder}")

remove_non_images("/content/drive/MyDrive/cloud_split/train")
remove_non_images("/content/drive/MyDrive/cloud_split/test")


Removed 0 non-image files from: /content/drive/MyDrive/cloud_split/train
Removed 0 non-image files from: /content/drive/MyDrive/cloud_split/test


In [ ]:
train_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    train_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    test_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical'
)


Found 5922 images belonging to 10 classes.
Found 2134 images belonging to 10 classes.


In [ ]:
history = model.fit(
    train_gen,
    validation_data=test_gen,
    epochs=5
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5
186/186 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9688 - loss: 0.0877

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


186/186 ━━━━━━━━━━━━━━━━━━━━ 243s 1s/step - accuracy: 0.9689 - loss: 0.0876 - val_accuracy: 0.9888 - val_loss: 0.0340
Epoch 2/5
186/186 ━━━━━━━━━━━━━━━━━━━━ 241s 1s/step - accuracy: 0.9766 - loss: 0.0659 - val_accuracy: 0.9906 - val_loss: 0.0266
Epoch 3/5
186/186 ━━━━━━━━━━━━━━━━━━━━ 241s 1s/step - accuracy: 0.9869 - loss: 0.0334 - val_accuracy: 0.9883 - val_loss: 0.0335
Epoch 4/5
186/186 ━━━━━━━━━━━━━━━━━━━━ 252s 1s/step - accuracy: 0.9493 - loss: 0.1489 - val_accuracy: 0.9892 - val_loss: 0.0264
Epoch 5/5
186/186 ━━━━━━━━━━━━━━━━━━━━ 248s 1s/step - accuracy: 0.9870 - loss: 0.0341 - val_accuracy: 0.9836 - val_loss: 0.0232


In [ ]:
import matplotlib.pyplot as plt

# Plot Accuracy
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid()
plt.show()

# Plot Loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid()
plt.show()

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

# Replace with your uploaded filename (check output of files.upload)
img_path = list(uploaded.keys())[0]

# Load image
img = image.load_img(img_path, target_size=(128, 128))
img_array = image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# Predict
pred = model.predict(img_array)
class_index = np.argmax(pred)
class_name = list(train_gen.class_indices.keys())[class_index]

# Show image and prediction
plt.imshow(img)
plt.axis('off')
plt.title(f"Prediction: {class_name}")
plt.show()

# Rain logic
rain_clouds = ['Cumulonimbus', 'Nimbostratus']
if class_name in rain_clouds:
    print("🌧️ It might rain.")
else:
    print("🌤️ It probably won't rain.")

In [ ]:
!pip install gradio

import gradio as gr

def predict_cloud(img):
    img_resized = img.resize((128, 128))
    img_array = np.array(img_resized) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    pred = model.predict(img_array)
    class_index = np.argmax(pred)
    class_name = list(train_gen.class_indices.keys())[class_index]

    rain_clouds = ['Cumulonimbus', 'Nimbostratus']
    message = "🌧️ It might rain." if class_name in rain_clouds else "🌤️ It probably won't rain."
    return class_name, message

gr.Interface(fn=predict_cloud,
             inputs=gr.Image(type="pil"),
             outputs=["text", "text"]).launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://cd621a8cdf9cae65c9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
model.save("cloud_classifier_model.keras")


In [ ]:
from google.colab import files
files.download("cloud_classifier_model.keras")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>